In [31]:
import pandas as pd

In [42]:
df1  = pd.read_csv("/home/harsh/Hackathons/Convolve/data/validation/validation_data_to_be_shared.csv")
# df2 = pd.read_csv("")

In [43]:
import pandas as pd

def find_uncommon_columns(file1_path, file2_path):
    """
    Compares two CSV files and identifies columns that are not common between them.

    Parameters:
    file1_path (str): Path to the first CSV file.
    file2_path (str): Path to the second CSV file.

    Returns:
    dict: A dictionary with keys 'uncommon_in_file1' and 'uncommon_in_file2',
          containing lists of columns that are unique to each file respectively.
    """
    try:
        # Load the CSV files
        df1 = pd.read_csv(file1_path)
        df2 = pd.read_csv(file2_path)

        # Get the column sets
        columns_file1 = set(df1.columns)
        columns_file2 = set(df2.columns)

        # Find uncommon columns
        uncommon_in_file1 = list(columns_file1 - columns_file2)
        uncommon_in_file2 = list(columns_file2 - columns_file1)

        return {
            'uncommon_in_file1': uncommon_in_file1,
            'uncommon_in_file2': uncommon_in_file2
        }
    except FileNotFoundError as e:
        print(f"Error: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
result = find_uncommon_columns("/home/harsh/Hackathons/Convolve/data/dev/Dev_data_to_be_shared.csv", "/home/harsh/Hackathons/Convolve/data/validation/validation_data_to_be_shared.csv")
print("Uncommon columns in file1:", result['uncommon_in_file1'])
print("Uncommon columns in file2:", result['uncommon_in_file2'])


Uncommon columns in file1: ['bad_flag']
Uncommon columns in file2: []


In [45]:
df1.shape

(41792, 1215)

In [46]:
from sklearn.preprocessing import LabelEncoder

In [47]:
def encode_categorical_columns(df, categorical_cols):
        binary_cols = [col for col in categorical_cols if df[col].nunique() == 2]
        for col in binary_cols:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])

        multi_cat_cols = [col for col in categorical_cols if df[col].nunique() > 2]
        df = pd.get_dummies(df, columns=multi_cat_cols, drop_first=True)
        return df


categorical_cols = df1.select_dtypes(include=['object', 'category']).columns.tolist()
if 'bad_flag' in categorical_cols:
    categorical_cols.remove('bad_flag')

df1 = encode_categorical_columns(df1, categorical_cols)

In [48]:
df1.shape

(41792, 1215)

In [49]:
def handle_missing_values(df, threshold=30000):
        col_missing_counts = df.isnull().sum()
        columns_to_drop = col_missing_counts[col_missing_counts > threshold].index
        df = df.drop(columns=columns_to_drop)
        return df
df1 = handle_missing_values(df1)

In [53]:
df1.shape

(41792, 1199)

In [50]:
numerical_cols = df1.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [51]:
len(numerical_cols)
from sklearn.impute import SimpleImputer

In [52]:
def impute_numerical_columns(df, numerical_cols):
        imputer = SimpleImputer(strategy='median')
        df[numerical_cols] = pd.DataFrame(
            imputer.fit_transform(df[numerical_cols]),
            columns=numerical_cols,
            index=df.index
        )
        return df

In [ ]:
df = impute_numerical_columns(df1, numerical_cols)

/home/harsh/Hackathons/Convolve/venv/lib/python3.12/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['bureau_436' 'bureau_447']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


ValueError: Shape of passed values is (41792, 1213), indices imply (41792, 1215)

In [5]:
import pandas as pd

# 1. Read two different CSV files with predictions
#    Each should have at least: "account_number" and "bad_flag" (or "predicted_bad_flag").
df1 = pd.read_csv("/home/harsh/Hackathons/Convolve/ensemble_predictions.csv")
df2 = pd.read_csv("/home/harsh/Hackathons/Convolve/predictions_output.csv")

print("df1 columns:", df1.columns.tolist())
print("df2 columns:", df2.columns.tolist())

df_merged = pd.merge(
    df1,
    df2,
    on="account_number",
    how="inner",
    suffixes=("_file1", "_file2")
)


# 3. Create a column to show if they have the same bad_flag
#    (This assumes both columns are named "bad_flag_file1" and "bad_flag_file2"
#    after the merge.)
df_merged["same_bad_flag"] = (
    df_merged["bad_flag_file1"] == df_merged["bad_flag_file2"]
).astype(int)

# 4. Count how many have the same bad_flag vs different
total_rows = len(df_merged)
num_same = df_merged["same_bad_flag"].sum()
num_different = total_rows - num_same

print(f"Total rows compared: {total_rows}")
print(f"Number with SAME bad_flag: {num_same}")
print(f"Number with DIFFERENT bad_flag: {num_different}")

# 5. (Optional) Inspect accounts that differ
df_diff = df_merged[df_merged["same_bad_flag"] == 0]
print("\nSample rows where the bad_flag doesn't match:")
print(df_diff.head())

# 6. (Optional) Save the merged comparison to a new CSV
df_merged.to_csv("comparison_of_predictions.csv", index=False)
print("\nMerged comparison file: comparison_of_predictions.csv")


df1 columns: ['account_number', 'onus_attribute_1', 'transaction_attribute_1', 'transaction_attribute_2', 'transaction_attribute_3', 'transaction_attribute_4', 'transaction_attribute_5', 'transaction_attribute_6', 'transaction_attribute_7', 'transaction_attribute_8', 'transaction_attribute_9', 'transaction_attribute_10', 'transaction_attribute_11', 'transaction_attribute_12', 'transaction_attribute_13', 'transaction_attribute_14', 'transaction_attribute_15', 'transaction_attribute_16', 'transaction_attribute_17', 'transaction_attribute_18', 'transaction_attribute_19', 'transaction_attribute_20', 'transaction_attribute_21', 'transaction_attribute_22', 'transaction_attribute_23', 'transaction_attribute_24', 'transaction_attribute_25', 'transaction_attribute_26', 'transaction_attribute_27', 'transaction_attribute_28', 'transaction_attribute_29', 'transaction_attribute_30', 'transaction_attribute_31', 'transaction_attribute_32', 'transaction_attribute_33', 'transaction_attribute_34', 'tran

/tmp/ipykernel_7905/3220480222.py:11: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  df_merged = pd.merge(


Total rows compared: 40956
Number with SAME bad_flag: 35389
Number with DIFFERENT bad_flag: 5567

Sample rows where the bad_flag doesn't match:
    account_number  onus_attribute_1_file1  transaction_attribute_1_file1  \
1           100420                     NaN                            NaN   
17          100436                 38000.0                            0.0   
18          100437                 65000.0                            0.0   
21          100440                 25000.0                            0.0   
22          100441                 27000.0                            0.0   

    transaction_attribute_2_file1  transaction_attribute_3_file1  \
1                             NaN                            NaN   
17                            0.0                            0.0   
18                            0.0                            0.0   
21                            0.0                            0.0   
22                            0.0                    